In [2]:
import pandas as pd

questions_df = pd.read_csv("../data/questions.csv")


In [10]:
e_q_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=0)
e_r_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=3)

s_q_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=1)
s_r_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=4)

g_q_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=2)
g_r_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=5)

In [36]:
import pandas as pd
import re

def transform_df(df, category):
    new_rows = []
    current_num = None
    current_question = None
    current_choices = []
    category_str = category

    def flush():
        if current_num is not None:
            int_current_num = int(current_num)
            new_rows.append({
                'question_number': int_current_num,
                'question': current_question,
                'choices': '\n'.join(current_choices),
                'category': category_str
            })

    for _, row in df.iterrows():
        number = row['번호']
        question_text = str(row['평가항목']) if pd.notna(row['평가항목']) else ""

        # 새로운 질문 발견 시 처리
        if pd.notna(number):
            flush()
            current_num = number
            current_question = question_text
            current_choices = []
        else:
            # 가. 나. 다. 형태의 choices 감지
            if re.match(r'^[가나다라마바사아자차카타파하]\.\s', question_text.strip()):
                current_choices.append(question_text.strip())

    flush()
    return pd.DataFrame(new_rows, columns=['question_number','question','choices', 'category'])

In [ ]:
def merge_ref_to_questions(df1, df2):
    grouped = df2.groupby('항목 번호').agg({
        '대분류': 'first',
        '중분류': 'first',
        '정의 및 확인사항 + 참고사례 및 제출 시 유의사항': lambda x: ' '.join(x.dropna())
    }).reset_index()

    grouped.rename(columns={'정의 및 확인사항 + 참고사례 및 제출 시 유의사항': '참고사항'}, inplace=True)

    merged_df = pd.merge(
        df1, 
        grouped, 
        left_on='question_number', 
        right_on='항목 번호', 
        how='left'
    )
    
    return merged_df.drop(columns=['항목 번호'])


In [56]:
e_questions_wich_choices = transform_df(e_q_df, "E")
merged_e_questions_wich_choices = merge_ref_to_questions(e_questions_wich_choices, e_r_df)

s_questions_wich_choices = transform_df(s_q_df, "S")
merged_s_questions_wich_choices = merge_ref_to_questions(s_questions_wich_choices, s_r_df)

g_questions_wich_choices = transform_df(g_q_df, "G")
merged_g_questions_wich_choices = merge_ref_to_questions(g_questions_wich_choices, g_r_df)

In [59]:
pd.concat([merged_e_questions_wich_choices, merged_s_questions_wich_choices, merged_g_questions_wich_choices]).to_csv("../data/questions_with_ref.csv", index=False)

In [ ]:
pd.read_csv("../data/questions_with_ref.csv")